# Fiber models - comparison of myelinated and unmyelinated fiber responses to gaussian acoustic source

### Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import logging

from PySONIC.utils import logger, si_format
from ExSONIC.core import SonicFiber, myelinatedFiberReilly, unmyelinatedFiberSundt, strengthDuration
from ExSONIC.plt import strengthDurationCurve
from ExSONIC.plot_SDcurves import plotSDcurves

#Folder used to store the strength duration data
outdir='strengthDurationData'
logger.setLevel(logging.WARNING)